In [1]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
from torchvision.datasets import FashionMNIST,StanfordCars
from matplotlib import pyplot as plt
import numpy as np
import torch.nn.functional as F
import math
import pandas as pd
from PIL import Image

In [2]:
import wandb

In [3]:
?wandb.init

In [4]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [5]:
# Defining the device

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return 
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = torch.tanh(out)

        return out

In [7]:
class Encoder(nn.Module):
    def __init__(self,block,layers,size):
        self.inplanes=64
        super(Encoder,self).__init__()
        self.conv1=nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3,bias=False)
        self.bn1=nn.BatchNorm2d(64)
        self.relu=nn.ReLU(inplace=True)
        self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)
        self.layer1=self._make_layer(block,16,layers[0])
        self.layer2 = self._make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)
#         self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(39168, size) ## CHANGE FROM HERE 
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    def forward(self,x):
        x=self.conv1(x)
        x=self.bn1(x)
        x=self.relu(x)
        
        x=self.maxpool(x)
        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
#         x=self.layer4(x)
        
        x=self.avgpool(x)
        x=x.view(x.size(0),-1)
        x=self.fc(x)
        
        return x

In [8]:
class Decoder(nn.Module):
    def __init__(self,size):
        super(Decoder,self).__init__()
        self.dfc3=nn.Linear(size,256)
        self.bn3=nn.BatchNorm1d(256)
        self.dfc2=nn.Linear(256,256)
        self.bn2=nn.BatchNorm1d(256)
        self.dfc1=nn.Linear(256,128*6*6)
        self.bn1=nn.BatchNorm1d(4608)
        self.upsample1=nn.Upsample(scale_factor=(2.073,2.4))
#         self.unflatten1 = nn.Unflatten(1, ())
        self.dconv5 = nn.ConvTranspose2d(128, 64, 3, padding = 0)
        self.dconv4 = nn.ConvTranspose2d(64, 32, 3, padding = 1)
        self.dconv3 = nn.ConvTranspose2d(32, 64, 3, padding = 1)
        self.dconv2 = nn.ConvTranspose2d(64, 32, 5, padding = 2)
        self.dconv1 = nn.ConvTranspose2d(32, 4, 24, stride = 4, padding = (10,12))
    def forward(self,x):
        x = self.dfc3(x)
        x = F.relu(self.bn3(x))
        
        x = self.dfc2(x)
        x = F.relu(self.bn2(x))
        x = self.dfc1(x)
        x = F.relu(self.bn1(x))
        x = x.view(-1,128,6,6)
        x=self.upsample1(x)
        x = self.dconv5(x)
        x = F.relu(x)
        x = F.relu(self.dconv4(x))
        x = F.relu(self.dconv3(x))
        x=self.upsample1(x)
        x = self.dconv2(x)
        x = F.relu(x)
        x=self.upsample1(x)

        x = self.dconv1(x)

        x = torch.sigmoid(x)
        return x

In [9]:
# tester=torch.ones((4,dims))
# decoder=Decoder(dims)
# (decoder(tester)).shape

In [10]:
class VAE(nn.Module):
    def __init__(self,dims):
        super(VAE,self).__init__()
        self.encoder = Encoder(Bottleneck, [3, 4, 6, 2],dims*2)
        self.decoder = Decoder(dims)
        self.dims=dims
    def reparameterise(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = std.data.new(std.size()).normal_()
            return eps.mul(std).add_(mu)
        else:
            return mu
    def forward(self,y):
        mu_logvar = self.encoder(y)
        mu_logvar=mu_logvar.view(-1,2,self.dims)
        mu = mu_logvar[:, 0, :]
        logvar = mu_logvar[:, 1, :]
        z = self.reparameterise(mu, logvar).view(-1,self.dims)
        z=self.decoder(z)
        return z,mu,logvar
        

In [11]:
dims=1024
model = VAE(dims).to(device)

In [12]:

# Setting the optimiser

learning_rate = 1e-3

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=learning_rate,
)

In [13]:
# Reconstruction + KL divergence losses summed over all elements and batch

def loss_function(ỹ, y, mu1, logvar1):
    BCE = nn.functional.binary_cross_entropy(
        ỹ, y, reduction='sum'
    )

    KLD = (-0.5 * torch.mean(-logvar1.exp() + logvar1 + 1.0 - mu1.pow(2)))
    print(KLD)
    return BCE+KLD,BCE,KLD

In [14]:
class MyDataset(Dataset):
    def __init__(self, train_path):
        self.df = pd.read_csv(train_path, sep=',', usecols=['input', 'output'])
    def __getitem__(self, index):
        x = np.array(Image.open(self.df.iloc[index, 1]))
        y = np.array(Image.open(self.df.iloc[index, 0]))
        x, y = torch.from_numpy(x), torch.from_numpy(y)
        return x, y

    def __len__(self):
#         return len(self.df)
        return 1000

In [15]:
epochs = 1000
batch_size = 4

In [16]:
wandb.config = {
  "learning_rate": learning_rate,
  "epochs": epochs,
  "batch_size": batch_size,
  "dims":dims
}

In [17]:
wandb.init(project="AerialPoseEstimator")

wandb: Currently logged in as: pthpth. Use `wandb login --relogin` to force relogin


In [18]:
train_loader=MyDataset("./dataset_train.csv")
test_loader=MyDataset("./dataset_test.csv")
train_loader=DataLoader(train_loader, batch_size=batch_size,shuffle=True)
test_loader=DataLoader(test_loader, batch_size=batch_size,shuffle=True)

In [19]:
wandb.watch(model)

[]

In [22]:
# def atch_mean_and_sd(loader):
    
#     cnt = 0
#     fst_moment = torch.empty(3)
#     snd_moment = torch.empty(3)

#     for images, _ in loader:
#         b, c, h, w = images.shape
#         nb_pixels = b * h * w
#         sum_ = torch.sum(images, dim=[0, 2, 3])
#         sum_of_square = torch.sum(images ** 2,
#                                   dim=[0, 2, 3])
#         fst_moment = (cnt * fst_moment + sum_) / (
#                       cnt + nb_pixels)
#         snd_moment = (cnt * snd_moment + sum_of_square) / (
#                             cnt + nb_pixels)
#         cnt += nb_pixels

#     mean, std = fst_moment, torch.sqrt(
#       snd_moment - fst_moment ** 2)        
#     return mean,std
def batch_mean(loader):
    

SyntaxError: invalid non-printable character U+00A0 (3982311932.py, line 2)

train_loader

In [20]:
# Training and testing the VAE

codes = dict(μ=list(), logσ2=list(), x=list())
for epoch in range(0, epochs + 1):
    # Training
    if epoch > 0:  # test untrained net first
        model.train()
        train_loss = 0
        bcs=0
        kls=0
        for x,y in train_loader:
            x = x.to(device)
            y = y.to(device)
            x=x.view(-1,3,480,720)
            y=y.view(-1,4,480,720)
            x=torch.div(x,255)
            y=torch.div(y,255)
            xs=[]
            ys=[]
            for i in x.split(360,-1):
                for j in i.split(240,-2):
                    xs.append(j)
            for i in y.split(360,-1):
                for j in i.split(240,-2):
                    ys.append(j)
            x=torch.cat(xs)
            y=torch.cat(ys)
            # ===================forward=====================
            y_bar, mu1, logvar1 = model(x)
            loss,bc,kl = loss_function(y_bar, y, mu1, logvar1)
            train_loss += loss
            bcs+=bc.item()
            kls+=kl
            # ===================backward====================
            optimizer.zero_grad()
            loss.backward()
#             print(loss.item())
            optimizer.step()
        # ===================log========================

    # Testing
        wandb.log({"train_loss":train_loss /len(train_loader.dataset), 
                       "train BCE Loss": bcs / len(train_loader.dataset),
                       "train KLD Loss": kls / len(train_loader.dataset)})
        means, logvars, labels = list(), list(), list()
        if epoch%10==0:
            torch.save({'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': train_loss,}, 
                       "./Weights/resnet.pt")
    torch.cuda.empty_cache()
    with torch.no_grad():
        model.eval()
        test_loss = 0
        bcs=0
        kls=0
        for x,y in test_loader:
            x = x.to(device)
            y = y.to(device)
            x=x.view(-1,3,480,720)
            y=y.view(-1,4,480,720)
            x=torch.div(x,255)
            y=torch.div(y,255)
            xs=[]
            ys=[]
            for i in x.split(360,-1):
                for j in i.split(240,-2):
                    xs.append(j)
            for i in y.split(360,-1):
                for j in i.split(240,-2):
                    ys.append(j)
            x=torch.cat(xs)
            y=torch.cat(ys)
            # ===================forward=====================
            ỹ, mu, logvar = model(x)
            # print(ỹ.shape)
            # print(y.shape)
            loss,bc,kl = loss_function(ỹ, y, mu, logvar)
            test_loss+=loss.item()
            bcs+=bc.item()
            kls+=kl.item()
    test_loss /= len(test_loader.dataset)
    wandb.log({"test_loss":test_loss /len(test_loader.dataset), 
                   "test BCE Loss": bcs / len(test_loader.dataset),
                   "test KLD Loss": kls / len(test_loader.dataset)})
    print(epoch)

tensor(0.0974, device='cuda:0')
tensor(0.0986, device='cuda:0')
tensor(0.0960, device='cuda:0')
tensor(0.0994, device='cuda:0')
tensor(0.0962, device='cuda:0')
tensor(0.0967, device='cuda:0')
tensor(0.0944, device='cuda:0')
tensor(0.0953, device='cuda:0')
tensor(0.0960, device='cuda:0')
tensor(0.0941, device='cuda:0')
tensor(0.0977, device='cuda:0')
tensor(0.0988, device='cuda:0')
tensor(0.0982, device='cuda:0')
tensor(0.0997, device='cuda:0')
tensor(0.0963, device='cuda:0')
tensor(0.0968, device='cuda:0')
tensor(0.0989, device='cuda:0')
tensor(0.0954, device='cuda:0')
tensor(0.0937, device='cuda:0')
tensor(0.0973, device='cuda:0')
tensor(0.0965, device='cuda:0')
tensor(0.0957, device='cuda:0')
tensor(0.0926, device='cuda:0')
tensor(0.0957, device='cuda:0')
tensor(0.0985, device='cuda:0')
tensor(0.0968, device='cuda:0')
tensor(0.0952, device='cuda:0')
tensor(0.0974, device='cuda:0')
tensor(0.0993, device='cuda:0')
tensor(0.0977, device='cuda:0')
tensor(0.0977, device='cuda:0')
tensor(0

tensor(0.4515, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.5671, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.7583, device='cuda:0', grad_fn=<MulBackward0>)
tensor(0.9147, device='cuda:0', grad_fn=<MulBackward0>)
tensor(1.5561, device='cuda:0', grad_fn=<MulBackward0>)
tensor(2.0182, device='cuda:0', grad_fn=<MulBackward0>)
tensor(2.8887, device='cuda:0', grad_fn=<MulBackward0>)
tensor(3.9636, device='cuda:0', grad_fn=<MulBackward0>)
tensor(3.3078, device='cuda:0', grad_fn=<MulBackward0>)
tensor(85.0255, device='cuda:0', grad_fn=<MulBackward0>)
tensor(7.5436, device='cuda:0', grad_fn=<MulBackward0>)
tensor(22.2954, device='cuda:0', grad_fn=<MulBackward0>)
tensor(16.7922, device='cuda:0', grad_fn=<MulBackward0>)
tensor(98.0832, device='cuda:0', grad_fn=<MulBackward0>)
tensor(50.2458, device='cuda:0', grad_fn=<MulBackward0>)
tensor(136.2183, device='cuda:0', grad_fn=<MulBackward0>)
tensor(28.6457, device='cuda:0', grad_fn=<MulBackward0>)
tensor(32.1419, device='cuda:0', grad_fn

tensor(7794.0591, device='cuda:0', grad_fn=<MulBackward0>)
tensor(32559.1289, device='cuda:0', grad_fn=<MulBackward0>)
tensor(639.2008, device='cuda:0', grad_fn=<MulBackward0>)
tensor(4791.6646, device='cuda:0', grad_fn=<MulBackward0>)
tensor(1379.5438, device='cuda:0', grad_fn=<MulBackward0>)
tensor(8770.8242, device='cuda:0', grad_fn=<MulBackward0>)
tensor(73.8636, device='cuda:0', grad_fn=<MulBackward0>)
tensor(71217.7031, device='cuda:0', grad_fn=<MulBackward0>)
tensor(2391.8689, device='cuda:0', grad_fn=<MulBackward0>)
tensor(422.8521, device='cuda:0', grad_fn=<MulBackward0>)
tensor(79.8772, device='cuda:0', grad_fn=<MulBackward0>)
tensor(423.5685, device='cuda:0', grad_fn=<MulBackward0>)
tensor(4635.9014, device='cuda:0', grad_fn=<MulBackward0>)
tensor(1808.2041, device='cuda:0', grad_fn=<MulBackward0>)
tensor(61703.9375, device='cuda:0', grad_fn=<MulBackward0>)
tensor(164.9377, device='cuda:0', grad_fn=<MulBackward0>)
tensor(313.3384, device='cuda:0', grad_fn=<MulBackward0>)
ten

tensor(228.0957, device='cuda:0')
tensor(4635.9790, device='cuda:0')
tensor(70724.0312, device='cuda:0')
tensor(162.1027, device='cuda:0')
tensor(890063.9375, device='cuda:0')
tensor(204.6973, device='cuda:0')
tensor(3297.7505, device='cuda:0')
tensor(7569.9453, device='cuda:0')
tensor(8739.6074, device='cuda:0')
tensor(122915.7656, device='cuda:0')
tensor(288.4722, device='cuda:0')
tensor(3029.2412, device='cuda:0')
tensor(1475.6866, device='cuda:0')
tensor(143.0294, device='cuda:0')
tensor(187.8914, device='cuda:0')
tensor(171357.5156, device='cuda:0')
tensor(512817.6562, device='cuda:0')
tensor(52893.6406, device='cuda:0')
tensor(2898281.2500, device='cuda:0')
tensor(100304.1562, device='cuda:0')
tensor(557392.5625, device='cuda:0')
tensor(224130.5469, device='cuda:0')
tensor(23360.1035, device='cuda:0')
tensor(2684.0684, device='cuda:0')
tensor(546.3204, device='cuda:0')
tensor(4663.7402, device='cuda:0')
tensor(209744.9219, device='cuda:0')
tensor(1046680.8125, device='cuda:0')
te

tensor(98.0205, device='cuda:0', grad_fn=<MulBackward0>)
tensor(4241.1440, device='cuda:0', grad_fn=<MulBackward0>)
tensor(2827.1055, device='cuda:0', grad_fn=<MulBackward0>)
tensor(186.4521, device='cuda:0', grad_fn=<MulBackward0>)
tensor(19853.0020, device='cuda:0', grad_fn=<MulBackward0>)
tensor(25568.5039, device='cuda:0', grad_fn=<MulBackward0>)
tensor(1099.4797, device='cuda:0', grad_fn=<MulBackward0>)
tensor(1815.6240, device='cuda:0', grad_fn=<MulBackward0>)
tensor(44.7879, device='cuda:0', grad_fn=<MulBackward0>)
tensor(629.0974, device='cuda:0', grad_fn=<MulBackward0>)
tensor(37.8343, device='cuda:0', grad_fn=<MulBackward0>)
tensor(6405.8955, device='cuda:0', grad_fn=<MulBackward0>)
tensor(63.5468, device='cuda:0', grad_fn=<MulBackward0>)
tensor(27502.8750, device='cuda:0', grad_fn=<MulBackward0>)
tensor(297.8167, device='cuda:0', grad_fn=<MulBackward0>)
tensor(2653.1204, device='cuda:0', grad_fn=<MulBackward0>)
tensor(573.7109, device='cuda:0', grad_fn=<MulBackward0>)
tensor

...

tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackw

tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, device='cuda:0')
tensor(nan, de

tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackward0>)
tensor(nan, device='cuda:0', grad_fn=<MulBackw

C:\Users\Frac36\AppData\Local\Temp\ipykernel_7700\4058169043.py:6: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'PngImageFile'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  y = np.array(Image.open(self.df.iloc[index, 0]))


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
test_loader=MyDataset("./dataset_test.csv")

In [ ]:
temp =np.array(Image.open("./Datasets/Input/Echendens-LHS_09620.png_6.png"), dtype = float)/255.0

In [ ]:
tem = torch.from_numpy(temp).view(-1,3,480,720)

In [ ]:
tem=tem.to(device,dtype=torch.float32)

In [ ]:
ans=(model(tem))

In [ ]:
ans=(ans[0]*255).detach().cpu().numpy()

In [ ]:
ans.shape

In [ ]:
img=Image.fromarray(ans)